# Importa as bibliotecas e as funções que serão necessárias para fazer os mapas mentais a partir de pdfs.

In [ ]:
import time
import google.generativeai as genai
import os
import time
import datetime
from IPython.display import Image, display
from dotenv import load_dotenv

### DEFINE FUNÇÃO QUE VERIFICA SE HÁ ARQUIVO ENVIADO POR UPLOAD À NUVEM GENAI
def verifica_arquivos_nuvem_genai():
    global varArquivoNaNuvem 
    lista_arquivos_nuvem = []
    for file in genai.list_files():
        lista_arquivos_nuvem.append(file)

    if not lista_arquivos_nuvem:
        print('Não há arquivos na nuvem Genai.')
        varArquivoNaNuvem = False
    else:
        print('Há arquivos na nuvem Genai.')
        print(genai.list_files())
        varArquivoNaNuvem = True

### DEFINE FUNÇÃO QUE VAI SOLICITAR A RESPOSTA DO GEMINI E FORMATÁ-LA EM PUML MINDMAP
def roda_API():
    global str_puml
    global lines
    response = model.generate_content([file, texto_prompt])
    # Procura garantir que a resposta do Gemini fique de fato no formato .puml:
    lines = response.text.split('\n')
    
    # Verificar se a primeira linha não começa com '*' nem '@startmindmap'
    if not lines[0].startswith('*') and not lines[0] == '@startmindmap':
        # Remover a primeira linha e adicionar '@startmindmap' na primeira linha
        lines = ['@startmindmap'] + lines[1:]
    
    # Verificar se a primeira linha começa com '*' 
    if lines[0].startswith('*'):
        # Acrescentar uma linha no início com '@startmindmap'
        lines = ['@startmindmap'] + lines
    
    # Verificar se a última linha não começa com '*' nem '@endmindmap'
    if not lines[-1].startswith('*') and not lines[-1] == '@endmindmap':
        # Remover a última linha e adicionar '@startmindmap' na última linha
        lines = lines[0:-1] + ['@endmindmap']
    
    # Verificar se a última linha começa com '*' 
    if lines[-1].startswith('*'):
        # Acrescentar uma linha no fim com '@endmindmap'
        lines = lines + ['@endmindmap']
    
    lines[1:-1] = [line for line in lines[1:-1] if line.startswith('*')]
            
    # Juntar as linhas novamente em uma string
    str_puml = '\n'.join(lines)
    # Confere se respeitou os criterios
    confere_criterio()

### DEFINE FUNÇÃO QUE VAI CONFERIR SE O PUML MINDMAP GERADO PELO GEMINI CHEGOU NO 4º NÍVEL, QUE É UMA LINHA INICIADA POR "**** "
def confere_criterio():
    global var_fim
    # Inicialização da variável contador_erro_criterio
    if 'contador_erro_criterio' not in globals():
        global contador_erro_criterio
        contador_erro_criterio = 0
    var_fim = False
    for line in lines:
        if line.startswith("**** "):
            var_fim = True
    if var_fim==True:
        gera_mapa_mental()
        contador_erro_criterio = 0
    else:
        print("O .puml gerado pelo Gemini não atende os critérios. Suspensão iniciada por 10 segundos...")
        time.sleep(10)
        print("10 segundos se passaram. Execução retomada. Vamos tentar gerar um mapa mental com o Gemini novamente.")
        contador_erro_criterio += 1
        if contador_erro_criterio < 5:
            roda_API()
        else:
            contador_erro_criterio = 0
            print(f"Não foi possível gerar mapa mental para o arquivo {file}. O Gemini gerou 5 vezes um .puml que não atendeu o critério de ter no mínimo 4 níveis.")

### DEFINE FUNÇÃO QUE VAI GERAR O MAPA MENTAL
def gera_mapa_mental():
    # Gera o mapa mental em formato .png
    print("Tentativa de gerar o mapa iniciada.")

    # Gerar a data e hora no formato desejado
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Nome do arquivo .puml com a data e hora
    path_puml = f"outputs\\{timestamp}_mapa_mental_{file.display_name}.puml"
    
    # Gerar o nome do arquivo PNG de saída
    path_png = path_puml[:-4]+"png"
    
    # Criar o arquivo PlantUML com o mapa mental
    with open(path_puml, "w") as f:
        f.write(str_puml)
    
    # Passo 2: Executar o PlantUML localmente para gerar a imagem
    plantuml_jar_path = "plantuml_jar\\plantuml-1.2024.7.jar"
    #command = f"java -jar \"{plantuml_jar_path}\" \"{path_puml}\""
    command = f"java -Dfile.encoding=UTF-8 -jar \"{plantuml_jar_path}\" \"{path_puml}\""
    
    # Executar o comando e verificar o retorno
    exit_code = os.system(command)
    
    if exit_code == 0:
        print("Imagem gerada com sucesso!")
    else:
        print("Houve um erro ao gerar a imagem.")
    
    # Passo 3: Exibir a imagem no Jupyter se ela foi gerada corretamente
    if os.path.exists(path_png):
        display(Image(path_png))
        contador_erros=0
    else:
        print("Imagem não encontrada. Verifique se o PlantUML converteu corretamente o arquivo .puml em .png.")
    # Suspender a execução por 10 segundos
    print("Suspensão de 10 segundos iniciada...")
    time.sleep(10)

# Inicializa do modelo Genai e faz upload dos pdfs para a nuvem Genai
Rode esse código para que os pdfs que estão na sua máquina, na pasta "pdf" deste projeto, sejam enviados para a nuvem Genai do Google.

In [ ]:
load_dotenv("keys.env")  # carrega variáveis do arquivo keys.env
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

#### CONFIGURA O LARGE LANGUAGE MODEL (LLM) GEMINI ####
genai.configure(api_key=GOOGLE_API_KEY) # Configura o genai com a GOOGLE_API_KEY
model = genai.GenerativeModel(model_name="gemini-1.5-flash") # Define o modelo Gemini.

verifica_arquivos_nuvem_genai() # Verifica se há arquivos na nuvem Genai

#### REMOVE TODOS OS ARQUIVOS QUE FORAM ENVIADOS POR UPLOAD À NUVEM GENAI (google.generativeai)  ####
if varArquivoNaNuvem:
    for file in genai.list_files(): # Inicia o loop na lista de arquivos da nuvem genai
        genai.delete_file(file.name) # Deletar arquivo da nuvem genai
        print(f"O arquivo {file.display_name} foi removido da nuvem.")

verifica_arquivos_nuvem_genai() # Verifica se há arquivos na nuvem Genai

#### FAZ UPLOAD DOS ARQUIVOS PARA A NUVEM GENAI (google.generativeai) ####
inputs_path = "inputs" # Define o diretório onde estão os arquivos PDF
arquivos = [os.path.join(inputs_path, pdf) for pdf in os.listdir(inputs_path)] # Lista todos os arquivos da inputs_path com caminho completo
arquivos.sort(key=os.path.getctime) # Ordena os arquivos pela data de criação
for path in arquivos: # Inicia o loop na lista dos caminhos completos dos arquivos
    nome_pdf = os.path.basename(path) # obtém o nome do arquivo
    if nome_pdf.endswith('.pdf'):  # Verifica se o arquivo é um PDF      
        file = genai.upload_file(path=path, display_name=nome_pdf)  # Faz upload do arquivo   
        print(f"Uploaded file '{file.display_name}' as: {file.uri}")
        time.sleep(1) # espera 1 segundo para iterar novamente, para facilitar a visualização de diferença de datas de criação dos arquivos na nuvem
uploaded_files = list(genai.list_files())  # cria a lista de uploads utilizando o generator genai.list_files()
sorted_files = sorted(uploaded_files, key=lambda file: file.create_time) # Ordena a lista de uploads pelo atributo 'create_time'
for file in sorted_files: # Exibe a lista ordenada de uploads
    print(f"{file.display_name} - {file.create_time}")

# Define o prompt que será enviado com cada pdf para o Gemini e executa todas as funções necessárias para gerar os mapas mentais em formato .png

In [ ]:
#### ENVIA OS PROMPTS PARA O GEMINI E GERA OS MAPAS MENTAIS ####
for file in sorted_files:
    global contador_erros
    contador_erros = 0
    lista_erros = []
    try:
        # Código que pode gerar um erro
        texto_prompt = '''
        Por favor, faça um mapa mental desse PDF em formato .puml de tópicos e subtópicos para ser usado no plantuml, sem indentações.
        Mantenha o idioma do PDF.
        O primeiro nível deve iniciar com '* '. O segundo nível deve iniciar com '** ', e assim sucessivamente.
        Não utilize '-' na parte inicial de cada tópico.
        O mapa deve ter pelo menos 4 níveis.
        No primeiro nível, são permitidas de 1 até 10 palavras.
        No segundo nível, são permitidas de 1 até 10 palavras.
        No terceiro nível, são permitidas de 1 até 10 palavras.
        No quarto nível, são permitidas de 1 até 10 palavras.
        Não inclua considerações finais, materiais complementares, referências bibliográficas ou outros tópicos que não explicitem conceitos.
        '''
        var_fim = False
        while var_fim == False:
            roda_API()

    except Exception as e:
        contador_erros+=1
        # Tratamento do erro (pode ser um log ou print)
        print(f"Ocorreu um erro com o item {file.display_name}: {e}")
        if contador_erros < 5:
            roda_API()
        else:
            lista_erros.append(file.display_name)
        # Passa para a próxima iteração
        continue
print("Script finalizado! Todos os arquivos da pasta 'pdf' foram varridos para gerar mapas mentais.")

# Imports the libraries and functions that will be necessary to create mind maps from PDFs.

In [ ]:
import time
import google.generativeai as genai
import os
import time
import datetime
from IPython.display import Image, display
from dotenv import load_dotenv

### DEFINES FUNCTION THAT CHECKS IF THERE ARE FILES UPLOADED TO THE GENAI CLOUD
def check_files_in_genai_cloud():
    global filesExistInCloud 
    cloud_file_list = []
    for file in genai.list_files():
        cloud_file_list.append(file)

    if not cloud_file_list:
        print('There are no files in the Genai cloud.')
        filesExistInCloud = False
    else:
        print('There are files in the Genai cloud.')
        print(genai.list_files())
        filesExistInCloud = True

### DEFINES FUNCTION THAT WILL REQUEST THE RESPONSE FROM GEMINI AND FORMAT IT IN PUML MINDMAP
def run_API():
    global str_puml
    global lines
    response = model.generate_content([file, prompt_text])
    # Tries to ensure that Gemini's response is indeed in .puml format:
    lines = response.text.split('\n')
    
    # Check if the first line doesn't start with '*' or '@startmindmap'
    if not lines[0].startswith('*') and not lines[0] == '@startmindmap':
        # Remove the first line and add '@startmindmap' as the first line
        lines = ['@startmindmap'] + lines[1:]
    
    # Check if the first line starts with '*' 
    if lines[0].startswith('*'):
        # Add a line at the beginning with '@startmindmap'
        lines = ['@startmindmap'] + lines
    
    # Check if the last line doesn't start with '*' or '@endmindmap'
    if not lines[-1].startswith('*') and not lines[-1] == '@endmindmap':
        # Remove the last line and add '@endmindmap' as the last line
        lines = lines[0:-1] + ['@endmindmap']
    
    # Check if the last line starts with '*' 
    if lines[-1].startswith('*'):
        # Add a line at the end with '@endmindmap'
        lines = lines + ['@endmindmap']
    
    lines[1:-1] = [line for line in lines[1:-1] if line.startswith('*')]
            
    # Join the lines back into a string
    str_puml = '\n'.join(lines)
    # Check if it met the criteria
    check_criteria()

### DEFINES FUNCTION THAT WILL CHECK IF THE PUML MINDMAP GENERATED BY GEMINI REACHED THE 4TH LEVEL, WHICH IS A LINE STARTING WITH "**** "
def check_criteria():
    global var_end
    # Initialization of the criteria_error_counter variable
    if 'criteria_error_counter' not in globals():
        global criteria_error_counter
        criteria_error_counter = 0
    var_end = False
    for line in lines:
        if line.startswith("**** "):
            var_end = True
    if var_end==True:
        generate_mind_map()
        criteria_error_counter = 0
    else:
        print("The .puml generated by Gemini does not meet the criteria. Suspension initiated for 10 seconds...")
        time.sleep(10)
        print("10 seconds have passed. Execution resumed. Let's try to generate a mind map with Gemini again.")
        criteria_error_counter += 1
        if criteria_error_counter < 5:
            run_API()
        else:
            criteria_error_counter = 0
            print(f"Unable to generate mind map for file {file}. Gemini generated a .puml that did not meet the criterion of having at least 4 levels 5 times.")

### DEFINES FUNCTION THAT WILL GENERATE THE MIND MAP
def generate_mind_map():
    # Generates the mind map in .png format
    print("Attempt to generate the map started.")

    # Generate the date and time in the desired format
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # .puml file name with date and time
    path_puml = f"outputs\\{timestamp}_mind_map_{file.display_name}.puml"
    
    # Generate the output PNG file name
    path_png = path_puml[:-4]+"png"
    
    # Create the PlantUML file with the mind map
    with open(path_puml, "w") as f:
        f.write(str_puml)
    
    # Step 2: Run PlantUML locally to generate the image
    plantuml_jar_path = "plantuml_jar\\plantuml-1.2024.7.jar"
    command = f"java -Dfile.encoding=UTF-8 -jar \"{plantuml_jar_path}\" \"{path_puml}\""
    
    # Execute the command and check the return
    exit_code = os.system(command)
    
    if exit_code == 0:
        print("Image generated successfully!")
    else:
        print("There was an error generating the image.")
    
    # Step 3: Display the image in Jupyter if it was generated correctly
    if os.path.exists(path_png):
        display(Image(path_png))
        error_counter=0
    else:
        print("Image not found. Check if PlantUML correctly converted the .puml file to .png.")
    # Suspend execution for 10 seconds
    print("10-second suspension initiated...")
    time.sleep(10)

# Initializes the Genai model and uploads PDFs to the Genai cloud
Run this code to send the PDFs from your machine, located in the "pdf" folder of this project, to Google's Genai cloud.

In [ ]:
load_dotenv("keys.env")  # loads variables from keys.env file
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

#### CONFIGURES THE LARGE LANGUAGE MODEL (LLM) GEMINI ####
genai.configure(api_key=GOOGLE_API_KEY) # Configures genai with GOOGLE_API_KEY
model = genai.GenerativeModel(model_name="gemini-1.5-flash") # Defines the Gemini model.

check_files_in_genai_cloud() # Checks if there are files in the Genai cloud

#### REMOVES ALL FILES THAT WERE UPLOADED TO THE GENAI CLOUD (google.generativeai) ####
if filesExistInCloud:
    for file in genai.list_files(): # Starts the loop in the list of files in the genai cloud
        genai.delete_file(file.name) # Delete file from genai cloud
        print(f"The file {file.display_name} was removed from the cloud.")

check_files_in_genai_cloud() # Checks if there are files in the Genai cloud

#### UPLOADS FILES TO THE GENAI CLOUD (google.generativeai) ####
inputs_path = "inputs" # Defines the directory where PDF files are located
files = [os.path.join(inputs_path, pdf) for pdf in os.listdir(inputs_path)] # Lists all files in inputs_path with full path
files.sort(key=os.path.getctime) # Sorts files by creation date
for path in files: # Starts the loop in the list of full file paths
    pdf_name = os.path.basename(path) # gets the file name
    if pdf_name.endswith('.pdf'):  # Checks if the file is a PDF      
        file = genai.upload_file(path=path, display_name=pdf_name)  # Uploads the file   
        print(f"Uploaded file '{file.display_name}' as: {file.uri}")
        time.sleep(1) # waits 1 second before iterating again, to facilitate viewing the difference in file creation dates in the cloud
uploaded_files = list(genai.list_files())  # creates the list of uploads using the genai.list_files() generator
sorted_files = sorted(uploaded_files, key=lambda file: file.create_time) # Sorts the list of uploads by the 'create_time' attribute
for file in sorted_files: # Displays the sorted list of uploads
    print(f"{file.display_name} - {file.create_time}")

# Defines the prompt that will be sent with each PDF to Gemini and executes all necessary functions to generate mind maps in .png format

In [ ]:
#### SENDS PROMPTS TO GEMINI AND GENERATES MIND MAPS ####
for file in sorted_files:
    global error_counter
    error_counter = 0
    error_list = []
    try:
        # Code that may generate an error
        prompt_text = '''
        Please create a mind map of this PDF in .puml format with topics and subtopics to be used in plantuml, without indentations.
        Maintain the language of the PDF.
        The first level should start with '* '. The second level should start with '** ', and so on.
        Do not use '-' at the beginning of each topic.
        The map should have at least 4 levels.
        At the first level, 1 to 10 words are allowed.
        At the second level, 1 to 10 words are allowed.
        At the third level, 1 to 10 words are allowed.
        At the fourth level, 1 to 10 words are allowed.
        Do not include final considerations, complementary materials, bibliographic references, or other topics that do not explicitly explain concepts.
        '''
        var_end = False
        while var_end == False:
            run_API()

    except Exception as e:
        error_counter += 1
        # Error handling (can be a log or print)
        print(f"An error occurred with item {file.display_name}: {e}")
        if error_counter < 5:
            run_API()
        else:
            error_list.append(file.display_name)
        # Move to the next iteration
        continue
print("Script finished! All files in the 'pdf' folder have been scanned to generate mind maps.")